# **Predictive Modeling for Rocket Landing Success**  
### *A Machine Learning Approach Using SpaceX Falcon 9 Data* 

## **Data Collection API**

### Installing Essential Python Libraries

**`requests`**: Simplifies sending HTTP requests.

**`pandas`**: Tools for data analysis and manipulation.

**`numpy`**: Library for working with arrays and performing mathematical operations.

In [ ]:
!pip install requests 
!pip install pandas
!pip install numpy

### Import Libraries

In [2]:
import requests
import pandas as pd
import numpy as np

import datetime
# Why No `pip install` for `datetime`?
# It's a standard Python library, which means it is included with Python by default. 
# There's no need to install it using `pip`, just import it directly and use it.


### Setting to Display All Columns And Data in a Dataframe

In [5]:
# Setting this option will print all columns of a dataframe
pd.set_option('display.max_columns', None)

# Setting this option will print all of the data in a feature
pd.set_option('display.max_colwidth', None)

### Define Auxiliary Functions that help us use the SpaceX API and extract data

In [9]:
# Request to the SpaceX API to Get Rocket data
response_test = requests.get("https://api.spacexdata.com/v4/rockets/")
print(response_test.json())

[{'height': {'meters': 22.25, 'feet': 73}, 'diameter': {'meters': 1.68, 'feet': 5.5}, 'mass': {'kg': 30146, 'lb': 66460}, 'first_stage': {'thrust_sea_level': {'kN': 420, 'lbf': 94000}, 'thrust_vacuum': {'kN': 480, 'lbf': 110000}, 'reusable': False, 'engines': 1, 'fuel_amount_tons': 44.3, 'burn_time_sec': 169}, 'second_stage': {'thrust': {'kN': 31, 'lbf': 7000}, 'payloads': {'composite_fairing': {'height': {'meters': 3.5, 'feet': 11.5}, 'diameter': {'meters': 1.5, 'feet': 4.9}}, 'option_1': 'composite fairing'}, 'reusable': False, 'engines': 1, 'fuel_amount_tons': 3.38, 'burn_time_sec': 378}, 'engines': {'isp': {'sea_level': 267, 'vacuum': 304}, 'thrust_sea_level': {'kN': 420, 'lbf': 94000}, 'thrust_vacuum': {'kN': 480, 'lbf': 110000}, 'number': 1, 'type': 'merlin', 'version': '1C', 'layout': 'single', 'engine_loss_max': 0, 'propellant_1': 'liquid oxygen', 'propellant_2': 'RP-1 kerosene', 'thrust_to_weight': 96}, 'landing_legs': {'number': 0, 'material': None}, 'payload_weights': [{'id'

#### Function to Get Booster Version from Dataset

This code block defines a function that is used to extract the booster name information from the `rocket` column in a dataset. The function works as follows:

1. **Iterates through each value in the `rocket` column**: For each rocket ID, it sends a request to the SpaceX API.
2. **Makes a request to the SpaceX API**: Uses the rocket identifier to fetch additional information about the rocket.
3. **Extracts the booster name**: The booster name is retrieved from the JSON response of the API.
4. **Appends the booster name to the list**: The booster name is added to the `BoosterVersion` list.

In [13]:
def getBoosterVersion(data):
    for x in data['rocket']:
        if x:
            reponse = requests.get("https://api.spacexdata.com/v4/rockets/"+str(x)).json()
            BoosterVersion.append(response['name'])


#### Function to Get Launch Site Information from Dataset

This code block defines a function that extracts launch site information using the `launchpad` identifier from the dataset. The function works as follows:

1. **Iterates through each value in the `launchpad` column**: For each launchpad ID, it sends a request to the SpaceX API.
2. **Makes a request to the SpaceX API**: Uses the launchpad identifier to fetch additional information about the launch site.
3. **Extracts relevant information**: The launch site name, longitude, and latitude are retrieved from the API's JSON response.
4. **Appends the data to the lists**: The launch site name, longitude, and latitude are added to the `LaunchSite`, `Longitude`, and `Latitude` lists, respectively.

In [12]:
def getLaunchSite(data):
    for x in data['launchpad']:
       if x:
         response = requests.get("https://api.spacexdata.com/v4/launchpads/"+str(x)).json()
         Longitude.append(response['longitude'])
         Latitude.append(response['latitude'])
         LaunchSite.append(response['name'])

### Function to Retrieve Rocket Core Data

This code block defines a function that extracts and stores additional data related to rocket cores from the SpaceX API. It uses the `cores` column from a dataset to retrieve detailed information about each rocket core. The function works as follows:

1. **Checks if a core ID exists**: If the core has a valid identifier (`core`), a request is made to the SpaceX API.
2. **Fetches core information**: The API is called to retrieve details about the core, such as the block, reuse count, and serial number.
3. **Appends data to lists**: The fetched data is added to the corresponding lists:
   - `Block`: The block number, indicating the core's version.
   - `ReusedCount`: The number of times the core has been reused.
   - `Serial`: The serial number of the core.
4. **Extracts additional information from the `cores` column**:
   - **Outcome**: The landing outcome (success and type).
   - **Flights**: The number of flights for that core.
   - **GridFins**: Whether gridfins (air control surfaces) were used.
   - **Reused**: Whether the core has been reused.
   - **Legs**: Whether legs were used for landing.
   - **LandingPad**: The identifier of the landing pad used.

In [15]:
# Takes the dataset and uses the cores column to call the API and append the data to the lists
def getCoreData(data):
    for core in data['cores']:
            if core['core'] != None:
                response = requests.get("https://api.spacexdata.com/v4/cores/"+core['core']).json()
                Block.append(response['block'])
                ReusedCount.append(response['reuse_count'])
                Serial.append(response['serial'])
            else:
                Block.append(None)
                ReusedCount.append(None)
                Serial.append(None)
            Outcome.append(str(core['landing_success'])+' '+str(core['landing_type']))
            Flights.append(core['flight'])
            GridFins.append(core['gridfins'])
            Reused.append(core['reused'])
            Legs.append(core['legs'])
            LandingPad.append(core['landpad'])